In [1]:
import pandas as pd
from hnmchallenge.data_reader import DataReader
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from hnmchallenge.evaluation.python_evaluation import map_at_k
from hnmchallenge.datasets.last_month_last_day import LMLDDataset
from hnmchallenge.constant import *

In [2]:
import logging
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [3]:
dataset = LMLDDataset()
dr = DataReader()

In [4]:
fd=dataset.get_full_data()

In [5]:
customers=dataset.get_customers_df()

In [8]:
customers=customers[[DEFAULT_USER_COL, "age"]]

In [9]:
customers

,customer_id,age
0,284440,49.0
1,26821,25.0
2,901651,24.0
3,1167050,54.0
4,1051706,52.0
...,...,...
1371975,470301,24.0
1371976,199407,21.0
1371977,111983,21.0
1371978,972397,18.0


In [11]:
fd2=pd.merge(fd, customers, on=DEFAULT_USER_COL, how='left')

In [24]:
fd3=fd2[fd2["age"]<=25]

In [30]:
fd3=fd2[fd2["age"]<=25].groupby([DEFAULT_ITEM_COL,"age"]).count().reset_index()


In [31]:
fd3= fd3[[DEFAULT_ITEM_COL,"age","t_dat"]].rename({"t_dat": "pop_25"}, axis=1)

In [33]:
fd3["pop_25"] = (fd3["pop_25"] - fd3["pop_25"].min()) / (fd3["pop_25"].max()- fd3["pop_25"].min())

In [34]:
fd3

,article_id,age,pop_25
0,0,16.0,0.000000
1,0,17.0,0.010086
2,0,18.0,0.029645
3,0,19.0,0.039731
4,0,20.0,0.069071
...,...,...,...
162154,26238,25.0,0.000000
162155,26243,21.0,0.000000
162156,26243,22.0,0.000306
162157,26245,21.0,0.000306


In [38]:
fd4=fd2[(fd2["age"]>25) & (fd2["age"]<=40)].groupby([DEFAULT_ITEM_COL,"age"]).count().reset_index()

In [39]:
fd4= fd4[[DEFAULT_ITEM_COL,"age","t_dat"]].rename({"t_dat": "pop_25_40"}, axis=1)

In [40]:
fd4["pop_25_40"] = (fd4["pop_25_40"] - fd4["pop_25_40"].min()) / (fd4["pop_25_40"].max()- fd4["pop_25_40"].min())

In [41]:
fd4

,article_id,age,pop_25_40
0,0,26.0,0.072384
1,0,27.0,0.053528
2,0,28.0,0.043491
3,0,29.0,0.040450
4,0,30.0,0.034367
...,...,...,...
305122,26242,38.0,0.000000
305123,26247,32.0,0.000000
305124,26248,36.0,0.000000
305125,26249,36.0,0.000304


In [42]:
fd5=fd2[(fd2["age"]>40) & (fd2["age"]<=60)].groupby([DEFAULT_ITEM_COL,"age"]).count().reset_index()

In [43]:
fd5= fd5[[DEFAULT_ITEM_COL,"age","t_dat"]].rename({"t_dat": "pop_40_60"}, axis=1)

In [45]:
fd5["pop_40_60"] = (fd5["pop_40_60"] - fd5["pop_40_60"].min()) / (fd5["pop_40_60"].max()- fd5["pop_40_60"].min())

In [46]:
fd5

,article_id,age,pop_40_60
0,0,41.0,0.026549
1,0,42.0,0.024779
2,0,43.0,0.046018
3,0,44.0,0.038938
4,0,45.0,0.038938
...,...,...,...
372354,26230,44.0,0.000000
372355,26234,46.0,0.000885
372356,26235,46.0,0.000000
372357,26244,59.0,0.000000


In [50]:
fd6=fd2[fd2["age"]>60]

In [51]:
fd6= fd6[[DEFAULT_ITEM_COL,"age","t_dat"]].rename({"t_dat": "pop_60"}, axis=1)

In [52]:
fd6["pop_60"] = (fd6["pop_60"] - fd6["pop_60"].min()) / (fd6["pop_60"].max()- fd6["pop_60"].min())

In [53]:
fd6

,article_id,age,pop_60
35,31,61.0,0.0
88,73,71.0,0.0
89,74,71.0,0.0
119,98,68.0,0.0
141,112,65.0,0.0
...,...,...,...
16020227,25585,69.0,1.0
16020274,8232,68.0,1.0
16020364,22823,68.0,1.0
16020382,19513,79.0,1.0


In [65]:
articles=dataset.get_articles_df()
articles=articles[DEFAULT_ITEM_COL]

In [66]:
articles=pd.merge(fd2,articles,on=DEFAULT_ITEM_COL,how="left")

In [67]:
articles

,t_dat,customer_id,article_id,price,sales_channel_id,age
0,2018-09-20,0,0,0.008458,2,22.0
1,2018-09-20,0,0,0.008458,2,22.0
2,2018-09-20,1,1,0.033881,1,26.0
3,2018-09-20,2,2,0.021593,1,53.0
4,2018-09-20,3,3,0.022017,2,38.0
...,...,...,...,...,...,...
16020499,2020-09-22,214284,24439,0.059305,2,32.0
16020500,2020-09-22,214284,19647,0.042356,2,32.0
16020501,2020-09-22,837397,25890,0.043203,1,67.0
16020502,2020-09-22,424416,6429,0.006763,1,21.0


In [68]:
articles= articles[articles.duplicated(subset=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])]

In [69]:
feature=pd.merge(articles,fd3,on=[DEFAULT_ITEM_COL,"age"],how="left")
feature=pd.merge(feature,fd4,on=[DEFAULT_ITEM_COL,"age"],how="left")
feature=pd.merge(feature,fd5,on=[DEFAULT_ITEM_COL,"age"],how="left")
feature=pd.merge(feature,fd6,on=[DEFAULT_ITEM_COL,"age"],how="left")

In [71]:
feature=feature[[DEFAULT_ITEM_COL,"age","pop_25","pop_25_40","pop_40_60","pop_60"]]

In [72]:
feature

,article_id,age,pop_25,pop_25_40,pop_40_60,pop_60
0,0,22.0,0.07335,NaN,NaN,NaN
1,3,38.0,NaN,0.010341,NaN,NaN
2,28,29.0,NaN,0.060827,NaN,NaN
3,37,33.0,NaN,0.027372,NaN,NaN
4,42,27.0,NaN,0.111922,NaN,NaN
...,...,...,...,...,...,...
4085012,23292,43.0,NaN,NaN,0.007080,NaN
4085013,24422,43.0,NaN,NaN,0.007965,NaN
4085014,25518,30.0,NaN,0.001825,NaN,NaN
4085015,3590,30.0,NaN,0.013382,NaN,NaN


In [48]:
fd4 = fd2.groupby([DEFAULT_ITEM_COL, "product_type_no"])["price"].min().reset_index(name="min")
fd4=pd.merge(fd3, fd4, on=[DEFAULT_ITEM_COL,"product_type_no"], how='left')

In [50]:
fd5 = fd2.groupby([DEFAULT_ITEM_COL, "product_type_no"])["price"].max().reset_index(name="max")
fd5=pd.merge(fd4, fd5, on=[DEFAULT_ITEM_COL,"product_type_no"], how='left')

In [51]:
fd5

,t_dat,customer_id,article_id,price,sales_channel_id,product_type_no,mean,min,max
0,2018-09-20,0,0,0.008458,2,253,0.007286,0.000203,0.008458
1,2018-09-20,0,0,0.008458,2,253,0.007286,0.000203,0.008458
2,2018-09-20,1,1,0.033881,1,272,0.031738,0.006763,0.033881
3,2018-09-20,2,2,0.021593,1,272,0.022026,0.006763,0.025407
4,2018-09-20,3,3,0.022017,2,286,0.019696,0.003373,0.022017
...,...,...,...,...,...,...,...,...,...
16020499,2020-09-22,214284,24439,0.059305,2,265,0.058314,0.029644,0.059305
16020500,2020-09-22,214284,19647,0.042356,2,-1,0.040636,0.016932,0.042356
16020501,2020-09-22,837397,25890,0.043203,1,66,0.049172,0.043203,0.050831
16020502,2020-09-22,424416,6429,0.006763,1,79,0.012916,0.004508,0.013542


In [53]:
fd5["price_mean"]=fd5["price"]/fd5["mean"]
fd5["price_min"]=fd5["price"]/fd5["min"]
fd5["price_max"]=fd5["price"]/fd5["max"]

In [54]:
fd5

,t_dat,customer_id,article_id,price,sales_channel_id,product_type_no,mean,min,max,price_mean,price_min,price_max
0,2018-09-20,0,0,0.008458,2,253,0.007286,0.000203,0.008458,1.160760,41.583333,1.000000
1,2018-09-20,0,0,0.008458,2,253,0.007286,0.000203,0.008458,1.160760,41.583333,1.000000
2,2018-09-20,1,1,0.033881,1,272,0.031738,0.006763,0.033881,1.067542,5.010025,1.000000
3,2018-09-20,2,2,0.021593,1,272,0.022026,0.006763,0.025407,0.980366,3.192982,0.849900
4,2018-09-20,3,3,0.022017,2,286,0.019696,0.003373,0.022017,1.117865,6.527638,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
16020499,2020-09-22,214284,24439,0.059305,2,265,0.058314,0.029644,0.059305,1.016993,2.000572,1.000000
16020500,2020-09-22,214284,19647,0.042356,2,-1,0.040636,0.016932,0.042356,1.042321,2.501502,1.000000
16020501,2020-09-22,837397,25890,0.043203,1,66,0.049172,0.043203,0.050831,0.878612,1.000000,0.849950
16020502,2020-09-22,424416,6429,0.006763,1,79,0.012916,0.004508,0.013542,0.523587,1.500000,0.499374


In [55]:
feature=fd5[[DEFAULT_ITEM_COL,"price_mean","price_min","price_max"]]

In [56]:
feature

,article_id,price_mean,price_min,price_max
0,0,1.160760,41.583333,1.000000
1,0,1.160760,41.583333,1.000000
2,1,1.067542,5.010025,1.000000
3,2,0.980366,3.192982,0.849900
4,3,1.117865,6.527638,1.000000
...,...,...,...,...
16020499,24439,1.016993,2.000572,1.000000
16020500,19647,1.042321,2.501502,1.000000
16020501,25890,0.878612,1.000000,0.849950
16020502,6429,0.523587,1.500000,0.499374


In [7]:
fd1=fd.groupby([DEFAULT_USER_COL, 't_dat']).size().reset_index(name='counts')

In [8]:
fd1

,customer_id,t_dat,counts
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2018-12-27,3
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-02,1
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-05-25,2
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-07-25,2
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2019-09-18,1
...,...,...,...
9080174,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-04-05,3
9080175,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-04-09,2
9080176,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-04-25,1
9080177,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,2020-06-22,1


In [10]:
fd2=fd1.groupby([DEFAULT_USER_COL])["counts"].mean().reset_index(name='average')

In [11]:
fd2

,customer_id,average
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,2.100000
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,3.739130
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,2.571429
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,2.000000
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,2.166667
...,...,...
1362276,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,4.636364
1362277,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,4.421053
1362278,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,2.368421
1362279,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,1.750000


In [12]:
intervals = [("2020-08-01", "2020-08-31"), ("2019-08-01", "2019-09-30"), ("2018-08-01", "2018-09-30")]

In [16]:
m = np.logical_or.reduce([np.logical_and(fd['t_dat'] >= l, fd['t_dat'] <= u)
                                    for l, u in intervals])
out = fd.loc[m]

In [17]:
out

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2
...,...,...,...,...,...
30990050,2020-08-31,fff7e7674509592818bf453391af43a85eaaac9a52d858...,0658030020,0.050831,1
30990051,2020-08-31,fff8722f6fe413c2df38cb09da2f458e5efe65cbcb3d08...,0923643001,0.025407,2
30990052,2020-08-31,fff8722f6fe413c2df38cb09da2f458e5efe65cbcb3d08...,0894320001,0.033881,2
30990053,2020-08-31,fffd176916f8c790ad54335c0f8478e613b5cb9b09ce2c...,0873678002,0.025407,1


In [79]:
df=fd.drop_duplicates(["t_dat",DEFAULT_USER_COL])

In [80]:
df

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,0,0,0.008458,2
2,2018-09-20,1,1,0.033881,1
3,2018-09-20,2,2,0.021593,1
4,2018-09-20,3,3,0.022017,2
6,2018-09-20,4,4,0.032186,2
...,...,...,...,...,...
14361074,2020-09-22,59779,20150,0.050831,2
14361083,2020-09-22,198005,20308,0.059305,2
14361085,2020-09-22,798833,21707,0.043203,1
14361086,2020-09-22,395317,4994,0.006763,1


In [81]:
df['date_diff'] = (df.pop('t_dat').groupby(df['customer_id']).diff().dt.days.fillna(0, downcast='infer'))

/tmp/ipykernel_23885/1392639203.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_diff'] = (df.pop('t_dat').groupby(df['customer_id']).diff().dt.days.fillna(0, downcast='infer'))


In [82]:
df

,customer_id,article_id,price,sales_channel_id,date_diff
0,0,0,0.008458,2,0
2,1,1,0.033881,1,0
3,2,2,0.021593,1,0
4,3,3,0.022017,2,0
6,4,4,0.032186,2,0
...,...,...,...,...,...
14361074,59779,20150,0.050831,2,17
14361083,198005,20308,0.059305,2,2
14361085,798833,21707,0.043203,1,21
14361086,395317,4994,0.006763,1,11


In [85]:
df[df[DEFAULT_USER_COL]==0]

,customer_id,article_id,price,sales_channel_id,date_diff
0,0,0,0.008458,2,0
3102169,0,0,0.007610,2,319
3265030,0,644,0.039576,2,11
4089565,0,1952,0.030492,2,51
8123440,0,1482,0.033881,2,192


In [83]:
user_average=df.groupby(DEFAULT_USER_COL)["date_diff"].mean().reset_index()

In [84]:
user_average

,customer_id,date_diff
0,0,114.600000
1,1,53.230769
2,2,67.125000
3,3,124.333333
4,4,0.000000
...,...,...
1136201,1136201,0.000000
1136202,1136202,0.000000
1136203,1136203,0.000000
1136204,1136204,0.000000


In [63]:
count_mb=fd.groupby(DEFAULT_USER_COL)["sales_channel_id"].size().reset_index()

In [64]:
count_mb

,customer_id,sales_channel_id
0,0,14
1,1,24
2,2,26
3,3,7
4,4,1
...,...,...
1136201,1136201,2
1136202,1136202,1
1136203,1136203,3
1136204,1136204,2


In [65]:
df=fd[fd["sales_channel_id"]==1]

In [66]:
count_mb1=df.groupby(DEFAULT_USER_COL)["sales_channel_id"].size().reset_index()

In [70]:
df.groupby(DEFAULT_USER_COL)["sales_channel_id"].size()

customer_id
1           7
2          26
5           2
6          18
7           9
           ..
1136190     6
1136194     7
1136196     1
1136197     2
1136202     1
Name: sales_channel_id, Length: 610744, dtype: int64

In [67]:
count=pd.merge(count_mb,count_mb1, on =DEFAULT_USER_COL, how='left')

In [68]:
count

,customer_id,sales_channel_id_x,sales_channel_id_y
0,0,14,NaN
1,1,24,7.0
2,2,26,26.0
3,3,7,NaN
4,4,1,NaN
...,...,...,...
1136201,1136201,2,NaN
1136202,1136202,1,1.0
1136203,1136203,3,NaN
1136204,1136204,2,NaN


In [74]:
count=count.fillna(0)

,customer_id,sales_channel_id_x,sales_channel_id_y,sales_channel_score
0,0,14,0.0,0.000000
1,1,24,7.0,0.291667
2,2,26,26.0,1.000000
3,3,7,0.0,0.000000
4,4,1,0.0,0.000000
...,...,...,...,...
1136201,1136201,2,0.0,0.000000
1136202,1136202,1,1.0,1.000000
1136203,1136203,3,0.0,0.000000
1136204,1136204,2,0.0,0.000000


In [72]:
count["sales_channel_score"]=count["sales_channel_id_y"]/count["sales_channel_id_x"]

In [73]:
count

,customer_id,sales_channel_id_x,sales_channel_id_y,sales_channel_score
0,0,14,NaN,NaN
1,1,24,7.0,0.291667
2,2,26,26.0,1.000000
3,3,7,NaN,NaN
4,4,1,NaN,NaN
...,...,...,...,...
1136201,1136201,2,NaN,NaN
1136202,1136202,1,1.0,1.000000
1136203,1136203,3,NaN,NaN
1136204,1136204,2,NaN,NaN


In [40]:
count_mb1

customer_id
1           7
2          26
5           2
6          18
7           9
           ..
1136190     6
1136194     7
1136196     1
1136197     2
1136202     1
Name: sales_channel_id, Length: 610744, dtype: int64

In [ ]:
fd[DEFAULT_USER_COL].nunique()

In [ ]:
fd = fd.drop_duplicates([DEFAULT_USER_COL, DEFAULT_ITEM_COL])

In [ ]:
fd

In [ ]:
count_mb = fd.groupby(DEFAULT_ITEM_COL).count()
feature = count_mb.reset_index()[[DEFAULT_ITEM_COL, "t_dat"]].rename(
            columns={"t_dat": "popularity"})

In [ ]:
feature

In [ ]:
feature["popularity_score"]=(feature["popularity"]-feature["popularity"].min())/(feature["popularity"].max()-feature["popularity"].min())

In [ ]:
feature

In [ ]:
feature['rank'] = feature['popularity_score'].rank(ascending=False,method='min').astype(int)

In [ ]:
feature

In [ ]:
df=dr.get_filtered_full_data()

In [ ]:
df= df.drop_duplicates([DEFAULT_USER_COL, DEFAULT_ITEM_COL])

In [ ]:
df

In [ ]:
df["last_buy"] = df.groupby(DEFAULT_USER_COL)["t_dat"].transform(max)
df["first_buy"] = df.groupby(DEFAULT_USER_COL)["t_dat"].transform(min)
df["time_score"] = (df["t_dat"] - df["first_buy"]) / (
            df["last_buy"] - df["first_buy"]
        )

In [ ]:
df=df.fillna(1)#df["tdiff"] =1/ ((df["last_buy"]-df["t_dat"]).dt.days) 

In [ ]:
df

In [ ]:
df["rank_time"] = df.groupby(DEFAULT_USER_COL)["time_score"].rank(ascending=False,method='min').astype(int)

In [ ]:
df

In [ ]:
df.loc[df[DEFAULT_USER_COL]==1]

In [ ]:
feature

In [ ]:
final=pd.merge(df, feature, on =DEFAULT_ITEM_COL, how='left')

In [ ]:
final

In [ ]:
final.loc[final[DEFAULT_USER_COL]==1]

In [ ]:
user=dr.get_filtered_full_data()

In [ ]:
user = user.drop_duplicates([DEFAULT_USER_COL])

In [ ]:
user['temp']=1

In [ ]:
user=user[[DEFAULT_USER_COL,'temp']]

In [ ]:
user

In [ ]:
feature_100=feature[feature['rank']<=100]

In [ ]:
feature_100['temp']=1

In [ ]:
final1=pd.merge(user, feature_100, on='temp')

In [ ]:
final1

In [ ]:
final1=final1.drop('temp', axis=1)

In [ ]:
final1

In [ ]:
final1[final1['customer_id']==1]

In [ ]:
final

In [ ]:
final=final.drop(['popularity_score','rank','popularity','t_dat','price','sales_channel_id','last_buy','first_buy'], axis=1)

In [ ]:
final2=pd.merge(final,final1,on=[DEFAULT_USER_COL,DEFAULT_ITEM_COL],how='outer')

In [ ]:
final[final[DEFAULT_USER_COL]==1]

In [ ]:
final2[final2[DEFAULT_USER_COL]==1]

In [ ]:
mask = final2.isnull().any(axis=1)

In [ ]:
final3=final2[~mask]

In [ ]:
final3[final3[DEFAULT_USER_COL]==1]

In [ ]:
final2=final2.fillna(0)

In [ ]:
final2

In [ ]:
final2=final2.drop(['rank_time','rank'],axis=1)

In [ ]:
a=0.95
eps=1e-6
final2['score']=a*final2['time_score']+(1-a+eps)*final2['popularity_score']

In [ ]:
final2

In [ ]:
final2["rank"] = final2.groupby(DEFAULT_USER_COL)["score"].rank(ascending=False,method='min').astype(int)

In [ ]:
final2

In [ ]:
final2=final2.drop(["time_score","popularity","popularity_score"],axis=1)

In [ ]:
final2[DEFAULT_USER_COL].nunique()

In [ ]:
final2

In [ ]:
cutoff = final2.groupby(DEFAULT_USER_COL).size().values

In [ ]:
cutoff

In [ ]:
i=0
filter_indices = []
for cut in cutoff:
    filter_indices.extend(range(i, i+12))
    i=i+cut

In [ ]:
filter_indices

In [ ]:
final2.head()

In [ ]:
final2 = final2.sort_values(DEFAULT_USER_COL, ascending=True)

In [ ]:
final2.head()

In [ ]:
final2 = final2.sort_values([DEFAULT_USER_COL, "rank"], ascending=True)

In [ ]:
final2

In [ ]:
final2 = final2.reset_index(drop=True)

In [ ]:
final_df = final2.loc[filter_indices]

In [ ]:
final_df[DEFAULT_USER_COL].nunique()

In [ ]:
final_final_df = final_df.drop(["score","rank"] ,axis=1)

In [ ]:
final_final_df[DEFAULT_USER_COL].nunique()

In [ ]:
from hnmchallenge.submission_handler import SubmissionHandler
sh = SubmissionHandler()     

In [ ]:
sh.create_submission_filtered_data([final_final_df], sub_name="Time_Pop")